# Generative netværk

Recurrent Neural Networks (RNNs) og deres gatede cellevarianter såsom Long Short Term Memory Cells (LSTMs) og Gated Recurrent Units (GRUs) gav en mekanisme til sproglig modellering, dvs. de kan lære ords rækkefølge og give forudsigelser for det næste ord i en sekvens. Dette gør det muligt for os at bruge RNNs til **generative opgaver**, såsom almindelig tekstgenerering, maskinoversættelse og endda billedbeskrivelse.

I RNN-arkitekturen, som vi diskuterede i den forrige enhed, producerede hver RNN-enhed den næste skjulte tilstand som output. Men vi kan også tilføje et andet output til hver rekurrent enhed, hvilket giver os mulighed for at generere en **sekvens** (som har samme længde som den oprindelige sekvens). Desuden kan vi bruge RNN-enheder, der ikke modtager input ved hvert trin, men blot tager en initial tilstandsvektor og derefter producerer en sekvens af outputs.

I denne notebook vil vi fokusere på simple generative modeller, der hjælper os med at generere tekst. For enkelhedens skyld lad os bygge et **karakter-niveau netværk**, som genererer tekst bogstav for bogstav. Under træning skal vi tage en tekstkorpus og opdele den i bogstavsekvenser.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## Opbygning af tegnordforråd

For at opbygge et generativt netværk på tegnniveau, skal vi opdele teksten i individuelle tegn i stedet for ord. Dette kan gøres ved at definere en anden tokenizer:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


Lad os se eksemplet på, hvordan vi kan kode teksten fra vores datasæt:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## Træning af en generativ RNN

Måden, vi vil træne en RNN til at generere tekst, er som følger. Ved hvert trin tager vi en sekvens af tegn med længden `nchars` og beder netværket om at generere det næste outputtegn for hvert inputtegn:

![Billede, der viser et eksempel på RNN-generering af ordet 'HELLO'.](../../../../../translated_images/da/rnn-generate.56c54afb52f9781d.webp)

Afhængigt af det konkrete scenarie kan vi også ønske at inkludere nogle specialtegn, såsom *end-of-sequence* `<eos>`. I vores tilfælde ønsker vi blot at træne netværket til uendelig tekstgenerering, så vi vil fastsætte størrelsen af hver sekvens til at være lig med `nchars` tokens. Derfor vil hvert træningseksempel bestå af `nchars` input og `nchars` output (hvilket er inputsekvensen forskudt med ét symbol til venstre). En minibatch vil bestå af flere sådanne sekvenser.

Måden, vi vil generere minibatches på, er ved at tage hver nyhedstekst med længden `l` og generere alle mulige input-output-kombinationer fra den (der vil være `l-nchars` sådanne kombinationer). Disse vil udgøre én minibatch, og størrelsen af minibatches vil variere ved hvert træningstrin.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

Lad os nu definere generatornetværket. Det kan baseres på enhver rekurrent celle, som vi diskuterede i den forrige enhed (simpel, LSTM eller GRU). I vores eksempel vil vi bruge LSTM.

Da netværket tager tegn som input, og ordforrådet er ret lille, behøver vi ikke en embedding-lag; one-hot-kodet input kan direkte sendes til LSTM-cellen. Dog, fordi vi sender tegnnumre som input, skal vi one-hot-kode dem, før de sendes til LSTM. Dette gøres ved at kalde funktionen `one_hot` under `forward`-passet. Output-encoderen vil være et lineært lag, der konverterer den skjulte tilstand til one-hot-kodet output.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

Under træning ønsker vi at kunne sample genereret tekst. For at gøre dette vil vi definere funktionen `generate`, som vil producere en output-streng af længden `size`, startende fra den indledende streng `start`.

Sådan fungerer det. Først vil vi sende hele start-strengen gennem netværket og tage output-tilstanden `s` og næste forudsagte tegn `out`. Da `out` er one-hot kodet, tager vi `argmax` for at få indekset for tegnet `nc` i ordforrådet og bruger `itos` til at finde det faktiske tegn og tilføje det til den resulterende liste af tegn `chars`. Denne proces med at generere ét tegn gentages `size` gange for at generere det ønskede antal tegn.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

Nu skal vi i gang med træningen! Træningsløkken er næsten den samme som i alle vores tidligere eksempler, men i stedet for nøjagtighed udskriver vi genereret tekst hver 1000. epoch.

Der skal lægges særlig vægt på, hvordan vi beregner tab. Vi skal beregne tabet ud fra den one-hot-kodede output `out` og den forventede tekst `text_out`, som er listen over tegnindekser. Heldigvis forventer funktionen `cross_entropy` et unormeret netværksoutput som første argument og klassens nummer som det andet, hvilket er præcis, hvad vi har. Den udfører også automatisk gennemsnit over minibatch-størrelsen.

Vi begrænser også træningen til `samples_to_train` prøver, for ikke at skulle vente for længe. Vi opfordrer dig til at eksperimentere og prøve længere træning, eventuelt over flere epochs (i så fald skal du oprette en ekstra løkke omkring denne kode).


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

Dette eksempel genererer allerede ret god tekst, men det kan forbedres yderligere på flere måder:

* **Bedre minibatch-generering**. Den måde, vi forberedte data til træning på, var at generere én minibatch fra én prøve. Dette er ikke ideelt, fordi minibatches har forskellige størrelser, og nogle af dem kan endda ikke genereres, fordi teksten er mindre end `nchars`. Derudover udnytter små minibatches ikke GPU'en tilstrækkeligt. Det ville være klogere at tage en stor tekstblok fra alle prøver, derefter generere alle input-output-par, blande dem og generere minibatches af ens størrelse.

* **Multilags LSTM**. Det giver mening at prøve 2 eller 3 lag af LSTM-celler. Som vi nævnte i den forrige enhed, udtrækker hvert lag af LSTM visse mønstre fra teksten, og i tilfælde af en generator på tegnniveau kan vi forvente, at det lavere LSTM-lag er ansvarligt for at udtrække stavelser, og de højere lag - for ord og ordkombinationer. Dette kan nemt implementeres ved at sende parameteren for antal lag til LSTM-konstruktøren.

* Du kan også eksperimentere med **GRU-enheder** og se, hvilke der fungerer bedre, samt med **forskellige størrelser på skjulte lag**. Et for stort skjult lag kan resultere i overtilpasning (f.eks. at netværket lærer den præcise tekst), og en mindre størrelse kan muligvis ikke producere gode resultater.


## Blød tekstgenerering og temperatur

I den tidligere definition af `generate` valgte vi altid det tegn med den højeste sandsynlighed som det næste tegn i den genererede tekst. Dette resulterede ofte i, at teksten "cyklede" mellem de samme tegnsekvenser igen og igen, som i dette eksempel:
```
today of the second the company and a second the company ...
```

Men hvis vi ser på sandsynlighedsfordelingen for det næste tegn, kan det være, at forskellen mellem de højeste sandsynligheder ikke er særlig stor, f.eks. kan ét tegn have en sandsynlighed på 0,2, mens et andet har 0,19 osv. For eksempel, når vi leder efter det næste tegn i sekvensen '*play*', kan det næste tegn lige så godt være et mellemrum eller **e** (som i ordet *player*).

Dette fører os til konklusionen, at det ikke altid er "retfærdigt" at vælge tegnet med den højeste sandsynlighed, fordi det at vælge det næsthøjeste stadig kan føre til meningsfuld tekst. Det er mere fornuftigt at **udvælge** tegn fra sandsynlighedsfordelingen givet af netværkets output.

Denne udvælgelse kan udføres ved hjælp af funktionen `multinomial`, som implementerer den såkaldte **multinomialfordeling**. En funktion, der implementerer denne **bløde** tekstgenerering, er defineret nedenfor:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

Vi har introduceret en ekstra parameter kaldet **temperatur**, som bruges til at angive, hvor strengt vi skal holde os til den højeste sandsynlighed. Hvis temperaturen er 1,0, udfører vi retfærdig multinomial sampling, og når temperaturen går mod uendelig - bliver alle sandsynligheder lige, og vi vælger tilfældigt den næste karakter. I eksemplet nedenfor kan vi observere, at teksten bliver meningsløs, når vi øger temperaturen for meget, og den minder om "cyklet" hårdt-genereret tekst, når den nærmer sig 0.



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi er ikke ansvarlige for eventuelle misforståelser eller fejltolkninger, der måtte opstå som følge af brugen af denne oversættelse.
